In [1]:
import os
import pandas as pd
import numpy as np
import pickle

from sklearn.utils.class_weight import compute_class_weight
from keras.utils import to_categorical

from keras.models import Sequential
from keras.layers import *

from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

import matplotlib.pyplot as plt
plt.style.use('seaborn')

Using TensorFlow backend.


In [2]:
EMBEDD_FILE = os.path.join("gensim", "embedd_weights.npy")
X_FILE_BIN =  os.path.join("gensim", "embedded_X.npy")
Y_FILE_BIN =  os.path.join("gensim", "encoded_Y.npy")

In [3]:
embedd_weights = np.matrix(np.load(EMBEDD_FILE))

In [4]:
embedd_weights.dtype

dtype('float32')

In [5]:
X = np.load(X_FILE_BIN)
Y = np.load(Y_FILE_BIN)

In [6]:
X.dtype

dtype('int32')

In [7]:
embedd_weights.shape, X.shape, Y.shape

((59717, 300), (838804, 679), (838804,))

In [8]:
NUM_WORDS = embedd_weights.shape[0]
EMBEDD_DIM = embedd_weights.shape[1]
PADDED_LEN = X.shape[1]

In [9]:
n_total = X.shape[0]
n_skip = 450_000 # 0
n_train = 500_000
n_val = 10_000 # 200_000
n_test = n_total - n_train - n_val

X_train = X[n_skip:n_train]
Y_train = to_categorical(Y[n_skip:n_train])

X_val = X[n_train:n_train+n_val]
Y_val = to_categorical(Y[n_train:n_train+n_val])

X_test = X[-n_test:]
Y_test = to_categorical(Y[-n_test:])

In [10]:
# compute class weights for keras (obtained from the whole of train sample)

classes = np.unique(Y[:n_train])
n_classes = len(classes)

class_weights = compute_class_weight('balanced', classes, Y[:n_train])  # ~ 1 / np.unique(Y, return_counts=True)[1]
class_weights

array([ 0.27602776,  0.72736371, 13.23801959,  9.49559404, 30.29201502,
        1.26825655])

In [11]:
from keras import backend as K

# Custom metrics.
# Note that any metric is computed per batch during training
# (hence one needs large batches for it to make sense).
# Also, there is the implicit 0.5 treshold in the K.round function below

def macroPrec(y_true, y_pred):
    
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)), axis=0)
    positives = K.sum(K.round(y_pred), axis=0)
    precision = true_positives / (positives + K.epsilon())

    macroPrec = K.mean( precision )
    
    return macroPrec


def macroRecall(y_true, y_pred):
    
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)), axis=0)
    actual_ones = K.sum(K.round(K.clip(y_true, 0, 1)), axis=0)
    recall = true_positives / (actual_ones + K.epsilon())

    macroRecall = K.mean( recall )
    
    return macroRecall


def macroF1(y_true, y_pred):
    
    def recall(y_true, y_pred):

        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)), axis=0)
        actual_ones = K.sum(K.round(K.clip(y_true, 0, 1)), axis=0)
        recall = true_positives / (actual_ones + K.epsilon())
        return recall

    
    def precision(y_true, y_pred):
        
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)), axis=0)
        positives = K.sum(K.round(K.clip(y_pred, 0, 1)), axis=0)
        precision = true_positives / (positives + K.epsilon())
        return precision
    
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    macroF1 = K.mean( 2*((precision*recall)/(precision+recall+K.epsilon())) )
    
    return macroF1



# Custom loss functions
# No clippping or rounding as those are not differentiable
# These functions are already taking into account class imbalances, so remeber
# not to set the class_weight argument in fitting method when using them as the loss 

def fuzzy_macroF1_flip(y_true, y_pred):
    
    def recall(y_true, y_pred):

        true_positives = K.sum(y_true * y_pred, axis=0)
        actual_ones = K.sum(y_true, axis=0)
        recall = true_positives / (actual_ones + K.epsilon())
        return recall

    
    def precision(y_true, y_pred):
        
        true_positives = K.sum(y_true * y_pred, axis=0)
        positives = K.sum(y_pred, axis=0)
        precision = true_positives / (positives + K.epsilon())
        return precision
    
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    macroF1 = K.mean( 2*((precision*recall)/(precision+recall+K.epsilon())) ) 
    
    return 1-macroF1

def w_cat_crossE(y_true, y_pred):
    
    class_entropy = K.sum(y_true * K.log(y_pred+K.epsilon()), axis=0)
    weighted_entropy = class_weights * class_entropy
    return -K.mean(weighted_entropy)/100

In [19]:
# some other global learning settings

batch_size = 2000
patience = 4
optimizer = 'adam' # 'rmsprop'
my_loss = fuzzy_macroF1_flip
my_metrics = ['categorical_accuracy', macroF1, w_cat_crossE] #, macroPrec, macroRecall]

In [20]:
# auxiliary functions
    
# to plot the learning history,
# i.e. loss and metrics on each train_batch and validation_batch
def plot_history(history):
    
    plt.figure(figsize=(18, 5))
    
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(loss) + 1)
    
    plt.subplot(1, 3, 1)    
    plt.plot(x, loss, 'ob', label='Training loss')
    plt.plot(x, val_loss, '-b', label='Validation loss')
    plt.title('Training and val losses') 
    plt.legend()
    
    all_metrics = np.array(list(set(history.history.keys()) - set(['loss', 'val_loss', 'lr'])))
    metrics = all_metrics[[not metric.startswith('val_') for metric in all_metrics]]
    plt.subplot(1, 3, 2)
    markers = ['b', 'r', 'g', 'y', 'c', 'm', 'k']
    marker = zip(markers, markers)
    for metric in metrics:
        results = history.history[metric]
        val_results = history.history['val_'+metric]
        mark1, mark2 = next(marker)
        plt.plot(x, results, 'o'+mark1, label=metric)
        plt.plot(x, val_results, '-'+mark2, label='Validation '+metric)
        plt.title('Training and val metrics')
    plt.legend()
    
    plt.subplot(1, 3, 3)
    lr = history.history['lr'] # learning rate
    plt.plot(x, lr, 'k', label='Learning rate')
    plt.title('Learning rate')  
    plt.legend()
    
    
# choose callback functions to be given in model.fit(...)
def callbacks(LRfactor=0.8):
    return [ReduceLROnPlateau(factor=LRfactor, patience = patience//2),
            EarlyStopping(monitor='val_macroF1', mode='max', patience=patience)
           ]

---

In [45]:
model = Sequential()
model.add(Embedding(input_dim=NUM_WORDS, output_dim=EMBEDD_DIM, input_length=PADDED_LEN))
model.add(Conv1D(48, 3))
model.add(MaxPooling1D(8, 4))
model.add(Conv1D(12, 4))
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 679, 300)          17915100  
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 677, 48)           43248     
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 168, 48)           0         
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 165, 12)           2316      
_________________________________________________________________
flatten_6 (Flatten)          (None, 1980)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 6)                 11886     
Total params: 17,972,550
Trainable params: 17,972,550
Non-trainable params: 0
________________________________________________________________

In [49]:
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=my_metrics)

In [50]:
history = model.fit(X_train,Y_train,
                    class_weight=class_weights,
                    epochs=30,
                    batch_size=batch_size,
                    validation_data=(X_val, Y_val),
                    callbacks=callbacks())

Train on 50000 samples, validate on 50000 samples
Epoch 1/30
50000/50000 [==============================] - 290s 6ms/step - loss: 0.6016 - categorical_accuracy: 0.7940 - macroF1: 0.3117 - val_loss: 0.3832 - val_categorical_accuracy: 0.8792 - val_macroF1: 0.4184
Epoch 2/30
50000/50000 [==============================] - 263s 5ms/step - loss: 0.3092 - categorical_accuracy: 0.9005 - macroF1: 0.4376 - val_loss: 0.3036 - val_categorical_accuracy: 0.9019 - val_macroF1: 0.4419
Epoch 3/30
50000/50000 [==============================] - 270s 5ms/step - loss: 0.2225 - categorical_accuracy: 0.9272 - macroF1: 0.4651 - val_loss: 0.2901 - val_categorical_accuracy: 0.9072 - val_macroF1: 0.4509
Epoch 4/30
50000/50000 [==============================] - 265s 5ms/step - loss: 0.1621 - categorical_accuracy: 0.9474 - macroF1: 0.5415 - val_loss: 0.2968 - val_categorical_accuracy: 0.9086 - val_macroF1: 0.4730
Epoch 5/30
50000/50000 [==============================] - 269s 5ms/step - loss: 0.1081 - categorical_a

---

In [24]:
model2 = Sequential()
model2.add(Embedding(input_dim=NUM_WORDS, output_dim=EMBEDD_DIM, input_length=PADDED_LEN,\
                     weights=[embedd_weights], trainable=False))
model2.add(Conv1D(48, 3))
model2.add(MaxPooling1D(8, 4))
model2.add(Conv1D(12, 4))
model2.add(Flatten())
model2.add(Dense(n_classes, activation='softmax'))
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 679, 300)          17915100  
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 677, 48)           43248     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 168, 48)           0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 165, 12)           2316      
_________________________________________________________________
flatten_4 (Flatten)          (None, 1980)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 11886     
Total params: 17,972,550
Trainable params: 57,450
Non-trainable params: 17,915,100
___________________________________________________________

In [25]:
model2.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=my_metrics)

In [26]:
model2.fit(X_train, Y_train,\
           class_weight=class_weights,
           epochs=30,
           batch_size=batch_size,
           validation_data=(X_val, Y_val),
           callbacks=callbacks())

Train on 50000 samples, validate on 10000 samples
Epoch 1/30
50000/50000 [==============================] - 150s 3ms/step - loss: 1.3390 - categorical_accuracy: 0.5838 - macroF1: 0.0602 - w_cat_crossE: 8.3672 - val_loss: 1.0983 - val_categorical_accuracy: 0.5967 - val_macroF1: 0.1257 - val_w_cat_crossE: 10.6121
Epoch 2/30
50000/50000 [==============================] - 149s 3ms/step - loss: 1.0736 - categorical_accuracy: 0.6024 - macroF1: 0.1254 - w_cat_crossE: 10.0598 - val_loss: 1.0647 - val_categorical_accuracy: 0.5967 - val_macroF1: 0.1245 - val_w_cat_crossE: 9.2511
Epoch 3/30
50000/50000 [==============================] - 148s 3ms/step - loss: 1.0587 - categorical_accuracy: 0.6024 - macroF1: 0.1253 - w_cat_crossE: 9.9541 - val_loss: 1.0527 - val_categorical_accuracy: 0.5967 - val_macroF1: 0.1245 - val_w_cat_crossE: 9.6521
Epoch 4/30
50000/50000 [==============================] - 148s 3ms/step - loss: 1.0415 - categorical_accuracy: 0.6024 - macroF1: 0.1253 - w_cat_crossE: 9.8550 - v

---

In [27]:
model3 = Sequential()
model3.add(Embedding(input_dim=NUM_WORDS, output_dim=EMBEDD_DIM, input_length=PADDED_LEN,\
                     weights=[embedd_weights], trainable=True))
model3.add(Conv1D(48, 3))
model3.add(MaxPooling1D(8, 4))
model3.add(Conv1D(12, 4))
model3.add(Flatten())
model3.add(Dense(n_classes, activation='softmax'))
model3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 679, 300)          17915100  
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 677, 48)           43248     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 168, 48)           0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 165, 12)           2316      
_________________________________________________________________
flatten_5 (Flatten)          (None, 1980)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 11886     
Total params: 17,972,550
Trainable params: 17,972,550
Non-trainable params: 0
________________________________________________________________

In [28]:
model3.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=my_metrics)

In [29]:
model3.fit(X_train, Y_train,\
           class_weight=class_weights,
           epochs=30,
           batch_size=batch_size,
           validation_data=(X_val, Y_val),
           callbacks=callbacks())

Train on 50000 samples, validate on 10000 samples
Epoch 1/30
50000/50000 [==============================] - 204s 4ms/step - loss: 1.1816 - categorical_accuracy: 0.5808 - macroF1: 0.0970 - w_cat_crossE: 9.1866 - val_loss: 0.8964 - val_categorical_accuracy: 0.5967 - val_macroF1: 0.1274 - val_w_cat_crossE: 8.7560
Epoch 2/30
50000/50000 [==============================] - 201s 4ms/step - loss: 0.5857 - categorical_accuracy: 0.7916 - macroF1: 0.3040 - w_cat_crossE: 7.5080 - val_loss: 0.4086 - val_categorical_accuracy: 0.8652 - val_macroF1: 0.4010 - val_w_cat_crossE: 6.1841
Epoch 3/30
14000/50000 [=======>......................] - ETA: 2:15 - loss: 0.3547 - categorical_accuracy: 0.8800 - macroF1: 0.4131 - w_cat_crossE: 5.7338

KeyboardInterrupt: 

---

In [26]:
# the workhorse

class BlackBox():
    
    def __init__(self, layers, loss, metrics, class_weight=None, callbacks=callbacks(), embedd_weights = None, embedd_dims=[None]*3):
        
        self.layers = layers
        self.loss = loss
        self.metrics = metrics
        self.class_weight = class_weight
        self.callbacks = callbacks
        self.history = None
        self.results = None
        
        self.model = Sequential()
        if embedd_weights.any():
            self.input_dim = embedd_dims[0] # num_words
            self.output_dim = embedd_dims[1] # dimension of the embedding space
            self.input_length = embedd_dims[2] # padded_length
            self.embedd_weights = embedd_weights
            
            self.model.add(Embedding(input_dim=self.input_dim,\
                                     output_dim=self.output_dim,\
                                     input_length=self.input_length,\
                                     weights=self.embedd_weights,\
                                     trainable=False))
            self.model.layers[0].trainable = False  # szczeżonego panbug szczeże
        for layer in layers:
            self.model.add(layer)
            
        self.model.add(Dense(n_classes, activation='softmax'))
        self.model.compile(loss=self.loss, optimizer=optimizer, metrics=self.metrics)
    
    def summary(self):
        return self.model.summary()
    
    def fit(self, epochs=200):
        self.history = self.model.fit(
            X_train, Y_train,
            class_weight=self.class_weight,
            epochs=epochs,
            batch_size=batch_size,
            validation_data=(X_val, Y_val),
            callbacks=self.callbacks,
            verbose = 1        
            )
        
    def reset(self):
        
        self.history = None
        self.results = None
        self.model.compile(loss=self.loss, optimizer=optimizer, metrics=self.metrics)
        
    def evaluate(self):
        print(f'Loss function: {self.loss.__name__ if callable(self.loss) else self.loss}. Metrics: {[metric.__name__ if callable(metric) else metric for metric in self.metrics]}')
        print("Predicting for X_test, comparing with y_test")
        return self.results
    
    def plot(self):
        plot_history(self.history)
        
    def Ksave(self, file):
        self.model.save(file)
        
    def save(self, file):
        pickle.dump(self, file)
 


In [31]:
# training an embedding, test run

layers1 = [GlobalAveragePooling1D()]

model1 = BlackBox(layers=layers1, loss=my_loss, metrics=my_metrics,\
                  embedd_weights=embedd_weights, embedd_dims=[NUM_WORDS, EMBEDD_DIM, PADDED_LEN])
model1.summary()

ValueError: Argument must be a dense tensor: (array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,